In [24]:
import pandas as pd
import requests
from itertools import cycle
import json
import csv
from urllib2 import Request, urlopen
from pandas.io.json import json_normalize

In [25]:
# Para compatibilidad entre múltiples plataformas
import os
IB = os.environ.get('INSTABASE_URI',None) is not None
open = ib.open if IB else open

In [26]:
#Abrir dataset
Dotacion = pd.read_csv('DOTACION.csv', delimiter = ";")
df = pd.DataFrame(Dotacion)
df.head(5)


,Periodo,ID RRHH,Rut,Rut_DV,Nombre Colaborador,Fecha de Nacimiento,ROL,Código Cargo,Descripcion Cargo,Agrupacion Cargo,...,Lugar Trabajo Actual,Nombre Lugar Trabajo,Lugar Trabajo Ant,Sexo,Centro Costo,Nombre Centro Costo,Orden Interna SAP,Clasificación Gasto,Segmento,Reemplazo Lic. Medica
0,201807,M26318,0116457954,11645795-4,"ABARCA CLAVERIA, RICHARD ANDRES",1970-09-20,GENERAL,48090,ASISTENTE DE OPERACIONES I,Asistente,...,1300270,CASA MATRIZ APOQUINDO,NaN,M,2007020500,JEFE DE OPERACIONES 4,CL05VS0181_A,OPEX,COL,NaN
1,201807,M19171,0117198928,11719892-8,"ABARCA CONTRERAS, ELIZABETH MARISOL",1970-07-17,GENERAL,48151,EJECUTIVO DE SERVICIO,Asistente de Servicio,...,500960,LOS ANDES AFP,50096.0,F,400010212,SERV. LOS ANDES AFP,CL08VP0044_V,VENTAS,COL,NaN
2,201807,M06087,012898609K,12898609-K,"ABARCA GALVEZ, MARICEL",1974-05-18,GENERAL,48043,ASISTENTE DE AREA,Asistente,...,1300270,CASA MATRIZ APOQUINDO,NaN,F,2006010000,SUBGERENCIA DE OPERACIONES,CL08VS0165_A,OPEX,COL,NaN
3,201807,M20266,0155603941,15560394-1,"ABARCA LOYOLA, MARCO ANDRES",1983-04-03,GENERAL,48151,EJECUTIVO DE SERVICIO,Asistente de Servicio,...,501191,VINA AFP,50119.0,M,400010233,COORDINADOR SERV AFP 3,CL08VP0067_V,VENTAS,COL,NaN
4,201807,M03243,0145249023,14524902-3,"ABARCA ORELLANA, MARIA PAZ",1976-12-03,PRIVADO,14840,SUBGERENTE DE OPERACIONES,Subgerente,...,1300270,CASA MATRIZ APOQUINDO,NaN,F,2006010000,SUBGERENCIA DE OPERACIONES,CL08VS0165_A,OPEX,GER,NaN


In [28]:
lista1 = df.Rut
nuevo=list()


for i in range(0, 1000):
    rut = lista1[i]
    #print rut
    
    #url = ['https://api.rutify.cl/rut/']
    #rut = [df.Rut]
    response = requests.get('https://api.rutify.cl/rut/' + rut)
    a = response.content.decode('utf-8')
    jason = json.loads(a)
    
    #data = json_normalize(jason)
    nuevo.append(jason)
    data = json_normalize(nuevo)
    #print nuevo
    
dt = pd.DataFrame(data)
dt.head(5)

ConnectionError: HTTPSConnectionPool(host='api.rutify.cl', port=443): Max retries exceeded with url: /rut/0153421498 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fba5d66c2d0>: Failed to establish a new connection: [Errno 101] Network is unreachable',))

In [18]:
#Direcccion Formateada google Maps
def get_lat_lng(address_string):

                url1="https://maps.googleapis.com/maps/api/geocode/json?address=%s" % (address_string)
                response = requests.get(url1, headers=USER_AGENT).json()
                result_list = list()
                for result in response['results']:
                    formatted_address = result['formatted_address']
                    lat = result['geometry']['location']['lat']
                    lng = result['geometry']['location']['lng']
                    result_list.append(formatted_address)
                    result_list.append(lat)
                    result_list.append(lng)
                    

                return result_list

In [22]:
lista = list()

USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

c = dt['servel.domicilio electoral'] + ', ' + dt['servel.provincia'] + ', ' + dt['servel.pais'] + ', ' + dt['servel.comuna']

for i in range (0, 5):
    Dir = c[i]
    #print Dir
    gm = get_lat_lng(Dir)
    lista.append(gm)
    #dt.append(lista)
    #dt.loc['Direccion'] = 0
    #data = json_normalize(lista)
    #print lista 
    
dt['Direccion Formateada'] = lista
dt.head(5)

,nombre,rut,servel.circunscripcion,servel.comuna,servel.domicilio electoral,servel.mesa,servel.pais,servel.provincia,servel.region,sexo,Direccion Formateada
0,abarca claveria richard andres,116457954,el cortijo,conchali,petropolis 1520 villa rep. del brasil,13 m,chile,santiago,metropolitana de santiago,1,[]
1,abarca contreras elizabeth marisol,117198928,san esteban,san esteban,av alessandri 1054,10 m,chile,los andes,valparaiso,2,"[Av. Alessandri 1054, San Esteban, Región de V..."
2,abarca galvez maricel elcira,12898609K,la granja,la granja,lirquen 7718 m concha,115 m,chile,santiago,metropolitana de santiago,2,"[Lirquén 7718, La Granja, Región Metropolitana..."
3,abarca loyola marco andres,155603941,peñalolen,peñalolen,2 oriente 2250 pq tobalaba,133 v,chile,santiago,metropolitana de santiago,1,"[Tobalaba 11855, Penalolen, Peñalolén, Región ..."
4,abarca orellana maria paz,145249023,san jose de la estrella,la florida,las amapolas 555,45 v,chile,santiago,metropolitana de santiago,2,"[Las Amapolas 555, La Florida, Región Metropol..."


In [23]:
dt.to_csv("DataFinal.csv", sep = ";", encoding = 'utf-8')